<a href="https://colab.research.google.com/github/robitussin/CCMACLRL_EXERCISES/blob/main/Exercise9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 9: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all classification models

Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e10/overview



In [75]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Dataset File

In [76]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/train.csv?raw=true'
df = pd.read_csv(dataset_url)

## Test File

In [77]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [78]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


## Sample Submission File

In [79]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [80]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           39098 non-null  int64  
 1   loan_status  39098 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 611.0 KB


Cleaning

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [82]:
df.isnull().sum()

id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64

In [83]:
label_encoder = LabelEncoder()
df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
dt['person_home_ownership'] = label_encoder.fit_transform(dt['person_home_ownership'])
     

df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])
dt['loan_intent'] = label_encoder.fit_transform(dt['loan_intent'])
     

df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])
dt['loan_grade'] = label_encoder.fit_transform(dt['loan_grade'])
     

df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])
dt['cb_person_default_on_file'] = label_encoder.fit_transform(dt['cb_person_default_on_file'])

In [84]:
df

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,3,0.0,1,1,6000,11.49,0.17,0,14,0
1,1,22,56000,2,6.0,3,2,4000,13.35,0.07,0,2,0
2,2,29,28800,2,8.0,4,0,6000,8.90,0.21,0,10,0
3,3,30,70000,3,14.0,5,1,12000,11.11,0.17,0,5,0
4,4,22,60000,3,2.0,3,0,6000,6.92,0.10,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,0,5.0,1,3,25000,15.95,0.21,1,10,0
58641,58641,28,28800,3,0.0,3,2,10000,12.73,0.35,0,8,1
58642,58642,23,44000,3,7.0,1,3,6800,16.00,0.15,0,2,1
58643,58643,22,30000,3,2.0,1,0,5000,8.90,0.17,0,3,0


In [85]:
x = df.drop(["loan_status", "id" ],axis=1)
y = df["loan_status"]

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1,test_size=0.3)

## 1. Train a KNN Classifier

In [86]:
score_list = {} #for saving score variables into it

In [87]:
KNN = KNeighborsClassifier(n_neighbors=22)
KNN.fit(x_train,y_train)
knn_score = KNN.score(x_test,y_test)

print(f"Score is {knn_score}")

Score is 0.8938274411731272


- Perform cross validation

In [88]:
scores = cross_val_score(KNN, x, y, cv=5)
scores

array([0.89453491, 0.89342655, 0.89342655, 0.89317077, 0.89359707])

In [89]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["KNN Classifier"] = scores.mean()

0.89 accuracy with a standard deviation of 0.00


## 2. Train a Logistic Regression Classifier

In [90]:
LR = LogisticRegression()
LR.fit(x_train,y_train)
lr_score = LR.score(x_test,y_test)

print(f"Score is {lr_score}")

Score is 0.8855291576673866


c:\Users\Neil\miniconda3\envs\COM222\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- Perform cross validation

In [91]:
scores = cross_val_score(LR, x, y, cv=5)
scores

c:\Users\Neil\miniconda3\envs\COM222\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Neil\miniconda3\envs\COM222\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

array([0.88353653, 0.88293972, 0.8831955 , 0.88208713, 0.88515645])

In [92]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["Logistic Regression"] = scores.mean()

0.88 accuracy with a standard deviation of 0.00


## 3. Train a Naive Bayes Classifier

In [52]:
NBC = GaussianNB()
NBC.fit(x_train,y_train)
NBC_score = NBC.score(x_test,y_test)

print(f"Score is {NBC_score}")

Score is 0.8824599295214277


- Perform cross validation

In [93]:
scores = cross_val_score(NBC, x, y, cv=5)
scores

array([0.87876204, 0.88149032, 0.88302498, 0.88242817, 0.88370705])

In [94]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["Naive Bayes"] = scores.mean()

0.88 accuracy with a standard deviation of 0.00


## 4. Train a SVM Classifier

In [55]:
svc = SVC()
svc.fit(x_train,y_train)
svc_score = svc.score(x_test,y_test)

print(f"Score is {svc_score}")

Score is 0.8579061043537569


- Perform cross validation

In [56]:
scores = cross_val_score(svc, x, y, cv=5)
scores

array([0.85761787, 0.85761787, 0.85761787, 0.85753261, 0.85761787])

In [57]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["SVM"] = scores.mean()

0.86 accuracy with a standard deviation of 0.00


## 5. Train a Decision Tree Classifier

In [95]:
dtc = DecisionTreeClassifier(random_state=1)
dtc.fit(x_train,y_train)

dtc_score = dtc.score(x_test,y_test)
print(f"Score is {dtc_score}")

Score is 0.9113334091167443


- Perform cross validation

In [96]:
scores = cross_val_score(dtc, x, y, cv=5)
scores

array([0.91457072, 0.90630062, 0.91039304, 0.9162759 , 0.9144002 ])

In [97]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["Decision Tree Classifiers"] = scores.mean()

0.91 accuracy with a standard deviation of 0.00


## 6. Train a Random Forest Classifier

In [98]:
rfc = RandomForestClassifier(n_estimators=50,random_state=1)
rfc.fit(x_train,y_train)
rfc_score = rfc.score(x_test,y_test)
score_list["RFC"]=rfc_score

print(f"Score is {rfc_score}")

Score is 0.9486188473343186


## 7. Compare all the performance of all classification models

In [99]:
score_list = list(score_list.items())

In [100]:
for alg,score in score_list:
    print(f"{alg} Score is {str(score)[:4]} ")

KNN Classifier Score is 0.89 
Logistic Regression Score is 0.88 
Naive Bayes Score is 0.88 
Decision Tree Classifiers Score is 0.91 
RFC Score is 0.94 


## 9. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [101]:
dt.drop('id', axis=1, inplace=True)

In [102]:
dt.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,23,69000,3,3.0,2,5,25000,15.76,0.36,0,2
1,26,96000,0,6.0,4,2,10000,12.68,0.10,1,4
2,26,30000,3,5.0,5,4,4000,17.19,0.13,1,2
3,33,50000,3,4.0,0,0,7000,8.90,0.14,0,7
4,26,102000,0,8.0,2,3,15000,16.32,0.15,1,4


In [103]:
id = sf.pop('id')
y_pred = rfc.predict(dt) #Used model is Random Forest

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'loan_status': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
